# Chapter 6 Design Patterns with First-Class Functions

Some design patterns may be either invisible or simpler in a dynamic language.

## Case Study: Refactoring Strategy

Class Inheritance based implementation

In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product: str, quantity: int, price: float):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self) -> float:
        return self.price * self.quantity

In [7]:
from __future__ import annotations
from typing import List

class Order:
    def __init__(self, customer: Customer, cart: List[LineItem], promotion:Promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self) -> float:
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self) -> float:
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        return '<Order total: {:.2f} due: {:2f}>'.format(self.total(), self.due())

Promotion strategies

In [9]:
class Promotion(ABC):
    @abstractmethod
    def discount(self, order: Order):
        """Return discount as a positive dollar amount"""
    
class FidelityPromo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order: Order) -> float:
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order: Order) -> float:
        return sum((item.total() * .1 for item in order.cart if item.quantity >= 20))
    
class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order: Order) -> float:
        distinct_items = set(map(lambda item: item.product, order.cart))
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

Usage or Order class with different promotions

In [15]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

print('Joe Fidelity:', Order(joe, cart, FidelityPromo()))
print('Ann Fidelity:', Order(ann, cart, FidelityPromo()))

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
print('Joe BulkItem:', Order(joe, banana_cart, BulkItemPromo()))
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print('Joe LargeOrder:', Order(joe, long_order, LargeOrderPromo()))

Joe Fidelity: <Order total: 42.00 due: 42.000000>
Ann Fidelity: <Order total: 42.00 due: 39.900000>
Joe BulkItem: <Order total: 30.00 due: 28.500000>
Joe LargeOrder: <Order total: 10.00 due: 9.300000>


### Function-Oriented Strategy

The `Promotion` has a single method `discount` and has no state. Therefore `Promotion`s can be replaced by functions.

In [16]:
from typing import Callable

class Order:
    def __init__(self, customer: Customer, cart: Order, promotion: Callable[[Order], float]=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
    
    def total(self) -> float:
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self) -> float:
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        return '<Order total: {:.2f} due: {:.2f}>'.format(self.total(), self.due())

In [20]:
def fidelity_promo(order: Order) -> float:
    """5% discount for customers with 1000 or more fidelity points""" 
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
def bulk_item_promo(order: Order) -> float:
    """10% discount for each LineItem with 20 or more units""" 
    return sum((item.total() * .1 for item in order.cart if item.quantity >= 20))
    
def large_order_promo(order: Order) -> float:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = set(map(lambda item: item.product, order.cart))
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [21]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

print('Joe Fidelity:', Order(joe, cart, fidelity_promo))
print('Ann Fidelity:', Order(ann, cart, fidelity_promo))

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
print('Joe BulkItem:', Order(joe, banana_cart, bulk_item_promo))
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print('Joe LargeOrder:', Order(joe, long_order, large_order_promo))

Joe Fidelity: <Order total: 42.00 due: 42.00>
Ann Fidelity: <Order total: 42.00 due: 39.90>
Joe BulkItem: <Order total: 30.00 due: 28.50>
Joe LargeOrder: <Order total: 10.00 due: 9.30>


### Finding Strategies in a Module

Filter names ending with `_promo`.

In [27]:
promos = [func for name, func in globals().items() if name.endswith('_promo') and name != 'best_promo']

def best_promo(order):
    return max(promo(order) for promo in promos)

print(promos)

[<function fidelity_promo at 0x7fc21040d400>, <function bulk_item_promo at 0x7fc1f69b7ea0>, <function large_order_promo at 0x7fc1f69b7840>]


In [31]:
print(best_promo(Order(joe, cart)))
print(best_promo(Order(joe, banana_cart)))
print(best_promo(Order(joe, long_order)))

0
1.5
0.7000000000000001
